In [1]:
# 过滤数据
df_2019_filtered = df_2019_1.filter(df_2019_1["trip_distance"] > 10) 
df_2019_filtered.show(5)

NameError: name 'df_2019_1' is not defined

df_2019_filtered.select(F.col('passenger_count') > 3).limit(10)

In [ ]:
df_2019_filtered.where(F.col('passenger_count') > 3).limit(10)

In [2]:
# 合并2019年和2021年的数据
 df_all = df_2019_1.union(df_2021_1)
 df_all.show(10)

IndentationError: unexpected indent (3649347377.py, line 2)

## Add new column ##

In [3]:
df_renamed = df_all.withColumnRenamed("RatecodeID", "Rate_codeID") 
df_renamed.show(5)

NameError: name 'df_all' is not defined

In [4]:
df_dropped = df_all.drop("passenger_count_plus_10")
df_dropped.show(5) 

NameError: name 'df_all' is not defined

In [5]:
df_all.groupBy("VendorID").agg({"passenger_count": "avg", "extra": "max"}).show()

NameError: name 'df_all' is not defined

## Sampling Data

In [6]:
SAMPLE_SIZE = 0.01

In [7]:
df = df_all.sample(SAMPLE_SIZE, seed = 20020223).toPandas() 
df

NameError: name 'df_all' is not defined

## Pre-Processing

In [8]:
df_2019_1.show(5)

NameError: name 'df_2019_1' is not defined

In [9]:
df_2019_1.printSchema() 
print(f"数据总量: {df_2019_1.count()}") 
df_2019_1.describe().show()

NameError: name 'df_2019_1' is not defined

## Handling missing data 

In [10]:
df_2019_1 = df_2019_1.dropna()
df_2019_1 = df_2019_1.fillna({'passenger_count': 1, 'trip_distance': 0.0})

NameError: name 'df_2019_1' is not defined

 ## Datatype Conversion

In [11]:
df_2019_1 = df_2019_1.withColumn("passenger_count", col("passenger_count").cast("integer"))
df_2019_1 = df_2019_1.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime")) 
df_2019_1 = df_2019_1.withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime"))  

NameError: name 'df_2019_1' is not defined

## Handling duplicates

In [12]:
df_2019_1 = df_2019_1.dropDuplicates()

NameError: name 'df_2019_1' is not defined

## Anomaly 

In [14]:
trip_distance_min = 0
trip_distance_max = 100
df_2019_1 = df_2019_1.filter((df_2019_1["trip_distance"] > trip_distance_min) & (df_2019_1["trip_distance"] > trip_distance_max))

NameError: name 'df_2019_1' is not defined

## Standardization 

In [15]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = VectorAssembler(inputCols=["trip_distance", "trip_duration"], outputCol= "features")
df_features = assembler.transform(df_2019_1)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features") 
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

AssertionError: 